# Graph embedding using SkipGram

This is an embedding of the whole graph, no training and validation split and all sources.

In [1]:
import sys
print(sys.path)

['/home/jtr4v/kg_covid_19_drug_analyses', '/home/jtr4v/anaconda3/lib/python38.zip', '/home/jtr4v/anaconda3/lib/python3.8', '/home/jtr4v/anaconda3/lib/python3.8/lib-dynload', '', '/home/jtr4v/anaconda3/lib/python3.8/site-packages', '/home/jtr4v/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/home/jtr4v/.ipython']


In [2]:
import silence_tensorflow.auto # Import needed to avoid TensorFlow warnings and general useless infos.

## Loading the graphs
We load the ppi graph from the repository as an undirected graph.

In [3]:
import urllib
import os
os.makedirs("whole_graph", exist_ok=True)
if not os.path.exists("whole_graph/kg-covid-19-skipgram-aug-2020.tar.gz"):
    with urllib.request.urlopen("https://zenodo.org/record/4011267/files/kg-covid-19-skipgram-aug-2020.tar.gz") as response, \
        open("graph/kg-covid-19-skipgram-aug-2020.tar.gz", 'wb') as out_file:
            data = response.read()  # a `bytes` object
            out_file.write(data)

In [4]:
import os
os.system("tar -xvzf whole_graph/kg-covid-19-skipgram-aug-2020.tar.gz -C whole_graph")

0

In [5]:
from ensmallen_graph import EnsmallenGraph

graph = EnsmallenGraph.from_csv(
    edge_path="whole_graph/merged-kg_edges.tsv",
    sources_column="subject",
    destinations_column="object",
    directed=False,
    default_edge_type="biolink:association",
    node_path="whole_graph/merged-kg_nodes.tsv",
    nodes_column="id",
    node_types_column="category",
    default_node_type="biolink:NamedThing",
    ignore_duplicated_edges=True,
    ignore_duplicated_nodes=True,
    force_conversion_to_undirected=True
)

As first thing, we print a short report showing all the avalable graph details, including the number of edges, nodes, trap nodes and both the connected components and the strongly connected components.

In [6]:
graph.report()

{'edges_number': '30861027',
 'selfloops_rate': '0.000015391581103247148',
 'is_directed': 'false',
 'density': '0.00021902960686152735',
 'degrees_mode': '1',
 'unique_node_types_number': '36',
 'degrees_median': '6',
 'strongly_connected_components_number': '8976',
 'unique_edge_types_number': '0',
 'traps_rate': '0.021906677500566116',
 'connected_components_number': '8976',
 'degrees_min': '0',
 'degrees_max': '90378',
 'nodes_number': '375365',
 'degrees_mean': '82.21604837957722',
 'singleton_nodes': '8223',
 'bidirectional_rate': '1'}

The followings are check that are not necessary, but are offered as sanity checks:

### Considered parameters
We are going to use the following parameters:

- **Walk lengths:** $100$ nodes.
- **Batch size:** $2^{7} = 128$ walks per batch.
- **Walk iterations:** $20$ iterations on the graph.
- **Window size:** $4$ nodes, meaning $4$ on the left and $4$ on the right of the center nodes. Consider that the first *window_size* values on the left and the right of the walks will be trimmed.
- **Return weight, inverse of $p$:** $1.0$.
- **Explore weight, inverse of $q$:** $1.0$.
- **Embedding size:** $100$.
- **Negative samples:** For the porpose of the [NCE function negative samples](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss), we are going to use $10$. These are the number of negative classes to randomly sample per batch. This single sample of negative classes is evaluated for each element in the batch.
- **Optimizer:** [Nadam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Nadam).
- **Early stopping parameters:** We are going to use an Early Stopping criterion on the *validation loss*, with patience $5$ and delta $0.0001$.
- **Epochs:** The model will be trained up to $1000$ epochs.
- **Learning rate:** since tipically the loss function is quite convex for the embedding problem, we can use a relatively higher learning rate. We are going to us $0.1$ for this example, to get to a faster convergence: this might lead to skipping some better minima that might be identified with a lower learning rate, such as the default one which is $0.0001$.

#### Setting up the parameters

In [15]:
walk_length=100
batch_size=2**8
iterations=20
window_size=4
p=1.0
q=1.0
embedding_size=100
negatives_samples=30
patience=5
delta=0.0001
epochs=1000
learning_rate=0.1

#### Creating the training and validation Keras sequences

In [16]:
from embiggen import Node2VecSequence

graph_sequence = Node2VecSequence(
    graph,
    walk_length=walk_length,
    batch_size=batch_size,
    iterations=iterations,
    window_size=window_size,
    return_weight=1/p,
    explore_weight=1/q
)

## Creating the SkipGram model
We are going to setup the model to use, if available, multiple GPUs.

In [17]:
from tensorflow.distribute import MirroredStrategy
from tensorflow.keras.optimizers import Nadam
from embiggen import SkipGram

strategy = MirroredStrategy()
with strategy.scope():
    model = SkipGram(
        vocabulary_size=graph.get_nodes_number(),
        embedding_size=embedding_size,
        window_size=window_size,
        negatives_samples=negatives_samples,
        optimizer=Nadam(learning_rate=learning_rate)
    )

model.summary()

Model: "SkipGram"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words_embedding (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 100)       37536500    words_embedding[0][0]            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 100)          0           embedding_2[0][0]                
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 8)]          0                                            
___________________________________________________________________________________________

## Tuning the SkipGram model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

history = model.fit(
    graph_sequence,
    steps_per_epoch=graph_sequence.steps_per_epoch,
    epochs=1000,
    callbacks=[
        EarlyStopping(
            "val_loss",
            min_delta=delta,
            patience=patience,
            restore_best_weights=True
        )
    ]
)

Epoch 1/1000
1435/1435 [==============================] - 677s 472ms/step - loss: 58.3536
Epoch 2/1000
1435/1435 [==============================] - 664s 463ms/step - loss: 35.8007
Epoch 3/1000
1435/1435 [==============================] - 667s 465ms/step - loss: 32.2220
Epoch 4/1000
1435/1435 [==============================] - 676s 471ms/step - loss: 27.3401
Epoch 5/1000
1435/1435 [==============================] - 678s 472ms/step - loss: 23.8743
Epoch 6/1000
1435/1435 [==============================] - 672s 468ms/step - loss: 22.7970
Epoch 7/1000
1435/1435 [==============================] - 668s 465ms/step - loss: 21.4614
Epoch 8/1000
1435/1435 [==============================] - 667s 465ms/step - loss: 19.5975
Epoch 9/1000
1435/1435 [==============================] - 658s 458ms/step - loss: 19.5827
Epoch 10/1000
1435/1435 [==============================] - 662s 461ms/step - loss: 18.9434
Epoch 11/1000
1435/1435 [==============================] - 665s 463ms/step - loss: 18.2576
Epoch 12

In [ ]:
### Saving the model weights
We save the obtained model weights:

In [ ]:
model.save_weights(f"{model.name}_whole_graph_weights.h5")

### Visualizing the training history
We can visualize the performance of the model during the training process as follows:

In [ ]:
from plot_keras_history import plot_history

plot_history(history)

There may be some hickups in the plot of the history if the model is reloaded from stored weights: [this is a known Keras issue](https://github.com/keras-team/keras/issues/4875) and is not related to either the holdouts used or the model.

## Saving the obtained embeddings
Finally we save our hard earned model embeddings. In another notebook we will show how to do link prediction on the obtained embedding.

In [ ]:
import numpy as np

np.save(f"{model.name}_whole_graph_embedding.npy", model.embedding)